### 第一部分：
- 基本統計量說明: Raw Data在當沖前/當沖後的各種基本統計量陳述（平均數、中位數、min、max、標準差），對Raw Data有個基本的概念

### 第二部分：
- 當沖前/當沖後的各種基本統計量的檢定（ex:現股當沖比重、日報酬率、週報酬率、月報酬率）
- 做平均數檢定、中位數檢定，看看當沖前/當沖後這些基本統計量有沒有顯著變化

### 第三部分：
- 政策面的研究。政府開放當沖是為了縮小spread、提高成交量、提高成交量週轉率。
- 研究data做的回歸是不是符合政府所宣稱的?當沖真的有穩定市場嗎？對資本市場有貢獻嗎？
- <span style="color:red">價量資料</span>

### 第四部分：
- 市場面研究。有沒有其他在市場上面的因素會影響我們的回歸式？要控制這些變數，放入回歸式（很多x)

#### stage point
1. 單向當沖基本統計量: 2013/6/13, 2014/1/6, 2014/6/30
2. 雙向當沖基本統計量(200間): 2014/1/6, 2014/6/30, 2014/12/31
3. 雙向當沖基本統計量(1432間): 2015/7/1, 2016/2/1, 2017/10/31
4. 稅率調降基本統計量(半年): 2016/11/1, 2017/4/28, 2017/10/31
5. 稅率調降基本統計量(一年): 2016/4/28, 2017/4/28, 2018/4/28

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import statsmodels.stats.api as sms
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings
import datetime as datetime
import json
import os
import csv
import math
from datetime import timedelta
from openpyxl import load_workbook
warnings.filterwarnings("ignore")
%matplotlib inline

from sklearn.linear_model import LinearRegression
from scipy import stats

In [3]:
class tw_day_trade():
    def __init__(self):
        self.pos_1 = ""
        self.pos_2 = ""
        self.pos_3 = ""
        self.df_before = "" 
        self.df_after = ""  
        self.basic_stats_info_before = []
        self.basic_stats_info_after = []
        self.basic_stats_info_ttest = []
        self.basic_stock_stats_info_dict = {}
        self.x = "" 
        self.y_daily = "" 
        self.y_weekly = "" 
        self.y_monthly = "" 
        self.day_trade_data = ""
        self.excel_data = {}

        """
        當沖相關重要日期們
        
        第一階段：2014/1/6 開放單向當沖200檔
        「台灣50、台灣中型100、富櫃50」200檔。
        
        第二階段：2014/6/30 開放雙向當沖
        仍為「台灣50、台灣中型100、富櫃50」200檔。
        
        第三階段：2015/6/1 新增「得發行認購(售)權證標的」及「ETF」亦得為當沖標的
        此時有377檔。
        
        第四階段：2016/2/1 開放「所有可做融資融券」的股票
        上市櫃共1432檔。
        
        第五階段：2017/4/28 調降當沖稅率
        """

    def read_raw_csv(self, str_filename):
        """
        讀進我們需要的raw data
        """
        self.day_trade_data = pd.read_csv(str_filename)
        print("Read Data Successfully")
        print("data shape: {}".format(self.day_trade_data.shape))

    def day_trade_split(self, date_1, date_2, date_3):
        """
        根據date_1, date_2, date_3來切割dataframe
        """
        tmp_arr = np.array(list(self.day_trade_data))
        self.pos_1 = np.where(tmp_arr==date_1)[0][0]
        self.pos_2 = np.where(tmp_arr==date_2)[0][0]
        self.pos_3 = np.where(tmp_arr==date_3)[0][0]
        #print("date_1 pos: {}\ndate_2 pos: {}\ndate_3 pos: {}".format(self.pos_1,self.pos_2,self.pos_3))
        
        self.df_before = self.day_trade_data.iloc[:,self.pos_1:self.pos_2]
        self.df_after = self.day_trade_data.iloc[:,self.pos_2:(self.pos_3+1)]
        #print("df_before shape: {}".format(self.df_before.shape))
        #print("df after shape: {}".format(self.df_after.shape))
        #print(self.day_trade_data)
        #print(self.df_before)
        #print(self.df_after)
          
    def day_trade_stats(self,flag):
        """
        第一部分：計算當沖前/當沖後的基本統計量，對Raw Data有個基本概念
        """
        
        # 平均數 (axis=1橫條往右apply func., axis=0是直條往下apply func.)
        # 取axis=1, 每間公司在一段期間內為一個單位
        self.df_before['mean_before'] = self.df_before.mean(axis=1)
        self.df_after['mean_after'] = self.df_after.mean(axis=1)
        self.df_before['std_before'] = self.df_before.std(axis=1)
        self.df_after['std_after'] = self.df_after.std(axis=1)
        self.df_before['median_before'] = self.df_before.median(axis=1)
        self.df_after['median_after'] = self.df_after.median(axis=1)
        
        # count, mean, std, min, 25%, 50%, 75%, max
        # 日/週/月報酬率/現股當沖比重
        if flag==0:
            for i in range(0,4):
                self.basic_stats_info_before.append(self.df_before['mean_before'][i::4].describe())
                self.basic_stats_info_after.append(self.df_after['mean_after'][i::4].describe())

        # 成交量/成交量週轉率
        if flag==1:
            self.basic_stats_info_before.append(self.df_before['mean_before'].describe())
            self.basic_stats_info_after.append(self.df_after['mean_after'].describe())
        
        # 賣價-買價
        if flag==2:
            for i in range(0,2):
                self.basic_stats_info_before.append(self.df_before['mean_before'][i::2].describe())
                self.basic_stats_info_after.append(self.df_after['mean_after'][i::2].describe())
        
        #print(self.df_before['mean_before'])
        #print(self.basic_stats_info_before)
        #print(self.basic_stats_info_after)
        
        print(self.basic_stats_info_ttest)
        #print(json.dumps(self.basic_stock_stats_info_dict, indent=2))

    def write_excel_reports_prepro(self,flag):
        arr_weight = []
        arr_day = []
        arr_week = []
        arr_month = []
        arr_vol = []
        arr_vol_turn = []
        arr_spread = [] #賣價-買價
        arr_spread_p = [] #(賣價-買價)/p
        
        self.excel_data = {
            'weight': arr_weight,
            'day': arr_day,
            'week': arr_week,
            'month': arr_month,
            'vol':arr_vol,
            'vol_turn':arr_vol_turn,
            'spread': arr_spread,
            'spread_p':arr_spread_p
        }
        
        if flag==0:
            for outside_item in range(len(self.basic_stats_info_before)):
                if outside_item == 0:# day
                    for j1 in range(1,len(self.basic_stats_info_before[0])):
                        arr_day.append(self.basic_stats_info_before[outside_item][j1])
                if outside_item == 1:# week
                    for j2 in range(1,len(self.basic_stats_info_before[0])):
                        arr_week.append(self.basic_stats_info_before[outside_item][j2])
                if outside_item == 2:# month
                    for j3 in range(1,len(self.basic_stats_info_before[0])):
                        arr_month.append(self.basic_stats_info_before[outside_item][j3])
                if outside_item == 3:# weight
                    for j4 in range(1,len(self.basic_stats_info_before[0])):
                        arr_weight.append(self.basic_stats_info_before[outside_item][j4])
            for outside_item in range(len(self.basic_stats_info_after)):
                if outside_item == 0:# day
                    for j1 in range(1,len(self.basic_stats_info_after[0])):
                        arr_day.append(self.basic_stats_info_after[outside_item][j1])
                if outside_item == 1:# week
                    for j2 in range(1,len(self.basic_stats_info_after[0])):
                        arr_week.append(self.basic_stats_info_after[outside_item][j2])
                if outside_item == 2:# month
                    for j3 in range(1,len(self.basic_stats_info_after[0])):
                        arr_month.append(self.basic_stats_info_after[outside_item][j3])
                if outside_item == 3:# weight
                    for j4 in range(1,len(self.basic_stats_info_after[0])):
                        arr_weight.append(self.basic_stats_info_after[outside_item][j4])

        if flag==1:
            for i in range(1,len(self.basic_stats_info_before[0])):
                arr_vol.append(self.basic_stats_info_before[0][i])
            
            for i in range(1,len(self.basic_stats_info_after[0])):
                arr_vol.append(self.basic_stats_info_after[0][i])
                
        # spread
        if flag==2:
            for outside_item in range(len(self.basic_stats_info_before)):
                if outside_item == 0: # spread
                    for j1 in range(1,len(self.basic_stats_info_before[0])):
                        arr_spread.append(self.basic_stats_info_before[outside_item][j1])
                if outside_item == 1: # spread/p
                    for j2 in range(1,len(self.basic_stats_info_before[0])):
                        arr_spread_p.append(self.basic_stats_info_before[outside_item][j2])
            
            for outside_item in range(len(self.basic_stats_info_after)):
                if outside_item == 0: # spread
                    for j1 in range(1,len(self.basic_stats_info_after[0])):
                        arr_spread.append(self.basic_stats_info_after[outside_item][j1])
                if outside_item == 1: # spread/p
                    for j2 in range(1,len(self.basic_stats_info_before[0])):
                        arr_spread_p.append(self.basic_stats_info_after[outside_item][j2])
        #print(json.dumps(self.excel_data, indent=2))
    
    def write_excel_reports(self,sheetName,types):
        """
        上市type=0, 上櫃type=1, 上市上櫃type=2
        """
        data = self.excel_data
        print(data)
        wb = load_workbook('20191007.xlsx')
        print(wb.get_sheet_names())

        itemList = ['B','C','D','E','F','G','H','I','J','K','L','M','N','O']
        rowIndex = -1
        columeIndex = -1
        result = True

        wb_sheet = wb[sheetName]
        print(wb_sheet)

        # 上市type=0, 上櫃type=1, 上市上櫃type=2
        for rowItem in data:

            if rowItem == 'weight':
                rowIndex = 5+types*15
            elif rowItem == 'day':
                rowIndex = 6+types*15
            elif rowItem == 'week':
                rowIndex = 7+types*15
            elif rowItem == 'month':
                rowIndex = 8+types*15
            elif rowItem == 'vol':
                rowIndex = 10+types*15
            elif rowItem == 'vol_turn':
                rowIndex = 11+types*15
            elif rowItem == 'spread':
                rowIndex = 13+types*15
            elif rowItem == 'spread_p':
                rowIndex = 14+types*15
            else:
                result = False
                break

            for columeIndex in range(0,len(data[rowItem])):
                wb_sheet[str(itemList[columeIndex])+str(rowIndex)] = round(data[rowItem][columeIndex],2)

        wb.save(r'20191007.xlsx')
    
    def day_trade_calculate(self):
        """
        準備OLS的x和y
        """
        self.x1 = self.df_after['mean_after'][3::4]-self.df_before['mean_before'][3::4]
        
        self.y_daily = self.df_after['std_after'][0::4]-self.df_before['std_before'][0::4]
        self.y_weekly = self.df_after['std_after'][1::4]-self.df_before['std_before'][1::4]
        self.y_monthly = self.df_after['std_after'][2::4]-self.df_before['std_before'][2::4]
    
    def day_trade_OLS_uni(self):
        y = self.y_monthly
        x = np.array(self.x1).reshape(-1,1)
        
        reg = LinearRegression().fit(x,y)
        print(reg.score(x, y))
        print(reg.intercept_)
        print(reg.coef_)
         
    def day_trade_OLS_multiple(self):
        y = self.y_daily
        x1 = np.array(self.x1).reshape(-1,1)
        x2 = np.array(np.log(pd.read_csv("new_data/資產總額_new/上市櫃20160428-20180428.csv",thousands=',')['2015/12/31'])).reshape(-1,1)
        x3 = np.array(pd.read_csv("new_data/DA_new/上市櫃2016.csv")).reshape(-1,1)
        x4 = np.array(pd.read_csv("new_data/PB_new/上市櫃2016.csv")).reshape(-1,1)
        x5 = np.array(pd.read_csv("new_data/beta_new/上市櫃2016.csv")).reshape(-1,1)
        
        print(len(x1))
        print(len(x2))
        print(len(x3))
        print(len(x4))
        print(len(x5))
        
        #x = sm.add_constant(x)
        #model = sm.OLS(y,x).fit()
        #print(model.summary())
    
            
    

In [7]:
data = tw_day_trade()
data.read_raw_csv("new_data/報酬率_pei/日報酬csv/上市上櫃公司20160428_20180428.csv")
data.day_trade_split("2016/4/28","2017/4/28","2018/4/27")
data.day_trade_stats(flag=0)
data.day_trade_calculate()
data.day_trade_OLS_multiple()

Read Data Successfully
data shape: (4004, 493)
[]
1001
1727
5181
5181
5181


UnboundLocalError: local variable 'x' referenced before assignment

<img src="https://i.imgur.com/M2y3Uha.jpg" height="500" width="500">
<img src="https://i.imgur.com/YZ8f21o.jpg" height="500" width="500">
<img src="https://i.imgur.com/VRp7NCJ.jpg" height="500" width="500">
<img src="https://i.imgur.com/XmJpppE.jpg" height="500" width="500">
<img src="https://i.imgur.com/7H8eg1g.jpg" height="500" width="500">

<img src="https://i.imgur.com/1giFKNu.png" height="800" width="800">